In [ ]:
# default_exp callbacks.buffer

# Buffer

> Callbacks for buffer

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.callbacks.core import *

In [ ]:
# export

class Buffer(Callback):
    def __init__(self, p_total):
        super().__init__(name='buffer', order=0)
        
        self.buffer = []
        self.p_total = p_total
        
    def __len__(self):
        return len(self.buffer)
    
    def add(self, item):
        
        if type(item)==list:
            for i in item:
                self.add(i)
        else:
            self.buffer.append(item)
                
    def setup(self):
        log = self.environment.log
        log.add_metric(f'diversity')
#         log.add_metric(f'valid')
        log.add_metric(f'bs')
            
    def sample(self, n):
        
        idxs = np.random.choice(np.arange(len(self.buffer)), min(n, len(self.buffer)), 
                                replace=False)
        batch = [self.buffer[i] for i in idxs]
        for idx in sorted(idxs, reverse=True):
            self.buffer.pop(idx)

        return batch
    
    def after_sample(self):
        env = self.environment
        batch_state = env.batch_state
            
        diversity = len(set(batch_state.samples))/len(batch_state.samples)
        self.environment.log.update_metric('diversity', diversity)
        self.environment.log.update_metric('bs', len(batch_state.samples))
        
    def after_build_buffer(self):
        if self.buffer:
            self.buffer = list(set(self.buffer))
    
    def sample_batch(self):
        env = self.environment
        batch_state = env.batch_state
        
        bs = int(env.bs * self.p_total)
        if bs>0:
            sample = self.sample(bs)
            batch_state.samples += sample
            batch_state.sources += ['buffer']*len(sample)

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()